## Ejercicio 3: Extraer entidades de un archivo de texto y devolver los resultados en formato JSON


La extracción de entidades es algo muy pesado y manual con un algoritmo estándar, pero con un LLM es un proceso súper eficiente y flexible. Vamos a extraer campos como nombres, números, emails y cuentas bancarias (si aparecen), estructurando el resultado en un .json para su uso posterior

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
API_VERSION = userdata.get('OPENAI_API_VERSION')
AZURE_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

print(type(OPENAI_API_KEY),OPENAI_API_KEY)
print(type(API_VERSION),API_VERSION)
print(type(AZURE_ENDPOINT),AZURE_ENDPOINT)

In [2]:
from openai import AzureOpenAI
import json

##### Establecemos el valor de la API key de OpenAI. En una aplicación real, no podemos exponer esta API key en el código, y la cogeríamos con una variable de entorno mediante un os.getenv()

##### Leemos el fichero de ejemplo

In [7]:
# Asegúrate de tener un archivo 'sample_text.txt' con el contenido a analizar.
try:
    with open("sample_text.txt", "r", encoding="utf-8") as f:
        texto_sample = f.read()
except FileNotFoundError:
    print("El archivo 'sample_text.txt' no se encontró. Por favor, crea el archivo y vuelve a ejecutar la celda.")
    texto_sample = ""

##### Definimos el prompt para la extracción de entidades

In [8]:
# Definimos el prompt para la extracción de entidades.
prompt_entidades = (
    "Eres un experto en procesamiento de lenguaje natural y extracción de información. "
    "Tu tarea es analizar el siguiente texto y extraer todas las entidades relevantes, organizándolas en categorías específicas. "
    "Debes identificar y extraer las siguientes entidades:\n\n"
    "1. Nombres: Nombres propios de personas, incluyendo clientes, empleados y cualquier otro responsable mencionado.\n"
    "2. Emails: Todas las direcciones de correo electrónico que aparezcan en el texto.\n"
    "3. Teléfonos: Números de teléfono, que pueden incluir guiones o espacios (ej. 600-123-456 o 600 123 456).\n"
    "4. Cuentas: Números de cuenta bancaria en formato IBAN simulado (ej. ES76 1234 5678 9012 3456 7890).\n"
    "5. Identificadores: Códigos de seguimiento, números de caso o cualquier identificador alfanumérico que haga referencia a un reporte o incidencia.\n"
    "6. Montos: Cifras en euros, expresadas con comas como separador decimal (ej. 500,00 euros), "
    "incluyendo el símbolo de euro o la palabra 'euros'.\n"
    "7. Fechas: Fechas en formato AAAA-MM-DD.\n\n"
    "Instrucciones adicionales:\n"
    "- Extrae únicamente las entidades que correspondan a cada categoría, sin incluir texto adicional o explicaciones.\n"
    "- Devuelve el resultado en formato JSON con la siguiente estructura EXACTA:\n\n"
    "{\n"
    '  "nombres": [lista de nombres],\n'
    '  "emails": [lista de emails],\n'
    '  "telefonos": [lista de teléfonos],\n'
    '  "cuentas": [lista de números de cuenta],\n'
    '  "identificadores": [lista de identificadores],\n'
    '  "montos": [lista de montos en euros],\n'
    '  "fechas": [lista de fechas]\n'
    "}\n\n"
    "Asegúrate de que cada lista contenga únicamente las entidades extraídas y no incluya duplicados. "
    "No agregues ningún comentario o texto adicional a la respuesta.\n\n"
    "Texto a analizar:\n"
    "<<TEXTO>>"
)

##### Realizamos la extracción:

In [ ]:
client = AzureOpenAI(
            api_key = OPENAI_API_KEY,
            api_version=API_VERSION,
            azure_endpoint= AZURE_ENDPOINT# Este endpoint corresponde a la región france-central
)

if texto_sample:
    prompt_entidades = prompt_entidades.replace("<<TEXTO>>", texto_sample)

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_entidades}],
        temperature=0.5,
        max_tokens=1000,
        response_format={"type": "json_object"},
    )

    salida_json_str = response.choices[0].message.content.strip()
    print("Salida JSON de entidades:")
    print(salida_json_str)

In [ ]:
print(type(salida_json_str))

##### Por último, convertimos la salida a un diccionario de Python:

In [ ]:
import pprint

try:
    entidades = json.loads(salida_json_str)
    print("\nJSON parseado:")
    pprint.pprint(entidades, compact=True)
except Exception as e:
    print("Error al parsear JSON:", e)

##### - ¿La extracción de entidades fue correcta? ¿Qué otras entidades podrían ser útiles en un entorno bancario?